# 02 - Створіть візуалізації, котрі відповідають на поставлені питання

Ваше завдання створити візуалізації, котрі відповідають на наступні питання:

- Як змінювалась структура генерації електроенергії за роками?

- Як залежить споживання електроенергії від дня року та години доби?

- Як змінюється генерація електроенергії з різних джерел впродовж доби?

- Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

- Як змінюється споживання електроенергії впродовж тижня?

Ви самостійно маєте обрати спосіб візуалізації. Ви також маєте написати короткий супровідний текст до кожної візуалізації, котрий пояснює, чому ви обрали саме цей спосіб презентації даних, які ще альтернативи ви розглядали, та чому зупинились саме на цьому варіанті (які його переваги та недоліки).

Джерело даних: Національна енергетична компанія "Укренерго"

## Підготовка даних

In [1]:
import pandas as pd
import altair as alt
import numpy as np

In [2]:
alt.data_transformers.enable('default', max_rows=None)

DataTransformerRegistry.enable('default')

In [3]:
df = pd.read_excel("2014-2020.xlsx")
df.head()

,Час/Дата,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD,Unnamed: 12
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693,-405.0,-84.0,-11.0,-29.0,NaN
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805,0.0,-43.0,-212.0,-46.0,NaN
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870,0.0,-13.0,-328.0,-30.0,NaN
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887,0.0,0.0,-64.0,-66.0,NaN
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387,0.0,-29.0,-48.0,-37.0,NaN


In [4]:
df.rename({'GAES_GEN': 'GAES'}, axis=1, inplace=True)

In [5]:
df['HOUR'] = df['Час/Дата'].str[:-11].astype(int)
df['HOUR'] = np.where(df['HOUR'] > 9, df['HOUR'], '0' + df['HOUR'].astype(str))
df['HOUR'] = np.where(df['HOUR'].astype(int) > 23, '00', df['HOUR'])

In [6]:
df['TIME'] = df['Час/Дата'].str[-4:] + '-' + df['Час/Дата'].str[-7:-5] + '-' + df['Час/Дата'].str[-10:-8] + ' ' + df['HOUR'].astype(str)

In [7]:
df['TIME'] = df['TIME'].astype('datetime64[ns]')

In [8]:
df = pd.melt(df, id_vars=['TIME', 'CONSUMPTION'], 
             value_vars=['AES', 'TEC', 'VDE', 'TES', 'GES', 'GAES'],
            var_name='STRUCT_NAME', value_name='STRUCT_VAL')

In [9]:
conditions = [
    (df['TIME'].dt.month == 1) | (df['TIME'].dt.month == 2) | (df['TIME'].dt.month == 12),
    (df['TIME'].dt.month == 3) | (df['TIME'].dt.month == 4) | (df['TIME'].dt.month == 5),
    (df['TIME'].dt.month == 6) | (df['TIME'].dt.month == 7) | (df['TIME'].dt.month == 8),
    (df['TIME'].dt.month == 9) | (df['TIME'].dt.month == 10) | (df['TIME'].dt.month == 11)
]

values = ['Winter', 'Spring', 'Summer', 'Autumn']

df['SEASON'] = np.select(conditions, values)

df.head()

,TIME,CONSUMPTION,STRUCT_NAME,STRUCT_VAL,SEASON
0,2020-12-31 00:00:00,16693,AES,9235.0,Winter
1,2020-12-31 23:00:00,17805,AES,9221.0,Winter
2,2020-12-31 22:00:00,18870,AES,9249.0,Winter
3,2020-12-31 21:00:00,19887,AES,9256.0,Winter
4,2020-12-31 20:00:00,20387,AES,9213.0,Winter


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368208 entries, 0 to 368207
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   TIME         368208 non-null  datetime64[ns]
 1   CONSUMPTION  368208 non-null  int64         
 2   STRUCT_NAME  368208 non-null  object        
 3   STRUCT_VAL   368207 non-null  float64       
 4   SEASON       368208 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 14.0+ MB


In [11]:
textgrey = '#333333'
ticksgray = '#999999'
lightgray = '#E7E7E7'

## Як змінювалась структура генерації електроенергії за роками?

Складові генерації: АЕС,ТЕЦ,ВДЕ,ТЕС,ГЕС,ГАЕС.

Для цієї візуалізації я хотіла використати або Bar Chart, або Line Chart (передати кількість згенерованої енергії за допопмогою ліній різних кольорів). Але все ж зупинилася на варіанті з Bar Chart, бо він може гарно передати відношення кількості згенерованої енергії з одного джерела до загальної кількості згенерованої за рік енергії. І зможемо побачити скільки загалом було згенеровано електроенергії за певний рік.

In [12]:
alt.Chart(df,
         title = 'Зміна структури генерації електроенергії за роками'
         ).mark_bar(size=60).encode(
    x = alt.X('year(TIME):O',
              title = None,
              axis = alt.Axis(labelAngle = 0,
                              labelAlign = 'center',
                              labelPadding = 10.0,
                              domain = False,
                              tickColor = lightgray,            
                             ),
            ),
    y = alt.Y('STRUCT_VAL:Q', 
              aggregate = 'sum',
              title = 'К-сть енергії в МВт',
              axis = alt.Axis(domain = False, 
                              labelPadding = 10.0, 
                              tickColor = lightgray,
                              titleColor = textgrey,
                             gridDash = [2, 2])
             ),
    color = alt.Color('STRUCT_NAME:N', title = 'Складові генерації'),
    tooltip = [
    alt.Tooltip(title = 'Складова', field = 'STRUCT_NAME', type = 'nominal'),
    alt.Tooltip(title = 'К-сть енергії в МВт', field = 'STRUCT_VAL', type = 'quantitative', aggregate = 'sum'),
    ]
).properties(width = 700, height = 500, background = '#F9F9F9', padding = 25
).configure_title(dx=0, 
                  dy=-20, 
                  color = textgrey, 
                  fontWeight=600, 
                  fontSize=14
)

Однак на такому графіку важко порівняти к-сть енергії з одного і того самого джерела за різні роки, тому я зробила ще одну візуалізацію, на якій якраз можна побачити і порівняти, як змінювалася к-сть енергії одного і того ж джерела протягом різних років.

In [13]:
alt.Chart(df,
         title = 'Зміна структури генерації електроенергії за роками'
         ).mark_bar().encode(
    x = alt.X('year(TIME):O',
              title = None,
              axis = alt.Axis(labelAngle = 0,
                              labelAlign = 'center',
                              labelPadding = 10.0,
                              domain = False,
                              tickColor = ticksgray
                             ),
             ),
    y = alt.Y('STRUCT_VAL:Q', 
              aggregate = 'sum',
              title = 'К-сть енергії в МВт',
              axis = alt.Axis(domain = False, 
                              labelPadding = 10.0, 
                              tickColor = ticksgray,
                              titleColor = textgrey,
                              gridDash = [2, 2]
                             )
             ),
    color = alt.Color('STRUCT_NAME:N', title = 'Складові генерації'),
    tooltip = [
    alt.Tooltip(title = 'Складова', field = 'STRUCT_NAME', type = 'nominal'),
    alt.Tooltip(title = 'К-сть енергії в МВт', field = 'STRUCT_VAL', type = 'quantitative', aggregate = 'sum'),
    ],
    column = alt.Column('STRUCT_NAME:N', title = None),
).properties(width = 750/len(pd.unique(df.STRUCT_NAME)), height = 500, background = '#F9F9F9', padding = 25
).configure_title(dx=340, dy=-20, 
                  color = textgrey, 
                  fontWeight=600, 
                  fontSize=14)

## Як залежить споживання електроенергії від дня року та години доби?

В цій візуалізації потрібно передати три величини, дві з яких час. Для цього може підійти heatmap. На вісях X та Y будуть часові проміжки, а кількість спожитої енерггії буде передаватися кольором.

Оскільки у таблиці присутні дані за декілька років, кожна позначка на графіку буде середнім арефметичним, характерним для певної години певного дня.

Мінус такого представлення в тому, що у нас буде 365 значень на осі Х, тому буде вадко роздивитися кожен окремий елемент, але ми зможемо подивитися на картину в цілому.

In [14]:
alt.Chart(df,
         title={
                "text": ["Залежність споживання електроенергії від дня року та години доби"], 
                "subtitle": ["(кожна позначка на графіку - середнє арефметичне за 2014-2020рр, характерне для певної години певного дня)"],
                }
         ).mark_rect().encode(
    alt.Y('hoursminutes(TIME):O', title=None,
          axis = alt.Axis(tickColor = lightgray, labelPadding = 5.0,),),
    alt.X('monthdate(TIME):O', title=None,
          axis = alt.Axis(ticks = False, labelPadding = 5.0,),
         ),
    alt.Color('CONSUMPTION:Q', 
              title='Споживання в МВт', 
              aggregate = 'average')
).properties(width = 800, height = 500, background = '#F9F9F9', padding = 25
).configure_title(dx=0, 
                  dy=-10, 
                  color = textgrey, 
                  fontWeight=600, 
                  fontSize=14)

## Як змінюється генерація електроенергії з різних джерел впродовж доби?

Для того щоб візуалізувати Як змінювалася генерація енергії з різних джерел впродовж доби, я також обрала BarChart. Як і в першому завданні було б зручно використати стовпчики, передати скільки взагалі генерується енергії в такий час доби, в які години всередньому генерується більше енергії, а в які менше. Кольори ж відображають кількість кожної окремої складової (AES, GAS ...),. Розмір кожного кольорового сегменту на стовпчику дозволяє порівняти відношення кількість кожної складової до загальної кількості згенерованої енергії за годину. А також ми можемо порівняти кожен елемент генерації між собою за різгні години. Хоча через зміщення кольорових сегментів це не дуже зручно. Це і є мінусом цього представлення.

Також я розглядала можливість використати Line Chart для цього представлення. Але на такому графіку було б важко порівняти як відноситься,наприклад, кількість згенерованої на Aes енергії до загальної кількості енергії згенерованої за певну годину дня. А також, ми б не бачили скільки загалом енергії генерується за певну годину.

In [15]:
alt.Chart(df,
         title = 'Зміна генерації електроенергії з різних джерел впродовж доби'
         ).mark_bar().encode(
    x = alt.X('hoursminutes(TIME):O',
              title = None,
              axis = alt.Axis(labelAngle = 0,
                              labelAlign = 'center',
                              labelPadding = 10.0,
                              domain = False,
                              ticks = False
              ),
             ),
    y = alt.Y('STRUCT_VAL:Q', 
              aggregate = 'average',
              title = 'К-сть енергії в МВт',
              axis = alt.Axis(domain = False, 
                              labelPadding = 10.0, 
                              tickColor = lightgray,
                              titleColor = textgrey,
                              gridDash = [2, 2])
             ),
    color = alt.Color('STRUCT_NAME:N', title = 'Складові генерації'),
    tooltip = [
    alt.Tooltip(title = 'Складова', field = 'STRUCT_NAME', type = 'nominal'),
    alt.Tooltip(title = 'К-сть енергії в МВт', field = 'STRUCT_VAL', type = 'quantitative', aggregate = 'average'),
    ],
).properties(width = 750, height = 500, background = '#F9F9F9', padding = 25
).configure_title(dx=0, 
      dy=-20, 
      color = textgrey, 
      fontWeight=600, 
      fontSize=14,
)

## Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

В даній візуалізації потрібно передати аж 4 величини. Я розбила це завдання на два графіки, перший стосується місяців, а другий пір року.

Оскільки вісі Х та Y будуть передавати часові проміжки, споживання електро енергії доведеться передавати в якийсь інший спосіб. Для цього чудово підійде HeatMap, на якій третя величина - 'споживання електроенергії', буде передаватися кольором. Чим більша кількість спожитої енергії, тим темнішим буде відповідний квадрат на графіку.

Мінус такого представлення - числові дані, а саме кількість  спожитої енергії, передаються дуже приблизно. Тобто навіть з присутністю Legend на якій описано якому діапазону кольорів відповідає який діапазон числових значень, буде важко визначити по кольору скільки там конкретно витраченої енергії. Глядачу може бути цікова дізнатися більш точне значення. На цей випадок я додала до графіку Tooltip. При на веденні на певний квадрат графіку, можна буде дізнатися точне числове значення якому він відповідає (дізнатися точну кількість спожитої енергії за певну годину певного місяця).

In [16]:
alt.Chart(df,
          title={
                "text": ["Споживання електроенергії впродовж доби у розрізі місяців року"], 
                "subtitle": ["(кожна позначка на графіку - середнє арефметичне за 2014-2020рр, характерне для певної години певного місяця)"],
                }
         ).mark_rect().encode(
    alt.Y('hoursminutes(TIME):O', title=None,
          axis = alt.Axis(labelPadding = 5.0,),
         ),
    alt.X('month(TIME):O',
          title = None,
          axis = alt.Axis(labelAngle = 0,
                          labelAlign = 'center',
                          labelPadding = 10.0,
                         ),
         ),
    alt.Color('CONSUMPTION:Q', title='consumption', aggregate = 'average')
).properties(width = 800, height = 500, background = '#F9F9F9', padding = 25
).configure_title(dx=0, 
      dy=-10, 
      color = textgrey, 
      fontWeight=600, 
      fontSize=14)

Наступний рафік більш узагальнена версія попереднього. Показує як змінюється споживання електроенергії залежно від години доби та  пори року.

In [17]:
alt.Chart(df, 
          title={
                "text": ["Споживання електроенергії впродовж доби у розрізі пір року"], 
                "subtitle": ["(кожна позначка на графіку - середнє арефметичне за 2014-2020рр, характерне для певної години певної пори року)"],
                }
         ).mark_rect().encode(
    alt.Y('hoursminutes(TIME):O', 
          title=None,
          
         ),
    alt.X('SEASON:N', 
          title=None, 
          sort=['Spring', 'Summer', 'Autumn', 'Winter'],
          axis = alt.Axis(labelAngle = 0,
                          labelAlign = 'center',
                          labelPadding = 10.0,
                          labelFontSize = 12.0
                         ),
         ),
    alt.Color('CONSUMPTION:Q', title='Споживання', aggregate = 'average'),
    tooltip = [
    alt.Tooltip(title = 'К-сть енергії в МВт', field = 'CONSUMPTION', type = 'quantitative', aggregate = 'average', format = '.4s'),
    ],
).properties(width = 800, height = 500, background = '#F9F9F9', padding = 25
).configure_title(dx=0, 
                  dy=-10, 
                  color = textgrey, 
                  fontWeight=600, 
                  fontSize=14,
)

## Як змінюється споживання електроенергії впродовж тижня?

Для цієї візуалізації я обрала Bar Chart, висота стовпчиків допоможе порівняти кількість споживання електроенергії протягом тижня.

Також я додала кольори, як додатковий спосіб порівняння. Кольори просто допомагають одразу виділити в який день тижня споживання в середньому найменше(Неділля), а в який - найбільше(Четвер).

In [18]:
alt.Chart(df, title = 'Споживання електроенергії впродовж тижня').mark_bar(size=60).encode(
    x = alt.X('day(TIME):O',
              title = None,
              axis = alt.Axis(labelAngle = 0,
                              labelAlign = 'center',
                              labelPadding = 10.0,
                              domain = False,
                              ticks = False
              ),
             ),
    y = alt.Y('CONSUMPTION:Q', 
              aggregate = 'average',
              title = 'Споживання енергії в МВт (середнє арифметичне)',
              axis = alt.Axis(domain = False, 
                              labelPadding = 10.0, 
                              tickColor = lightgray,
                              titleColor = textgrey,
                              gridDash = [2, 2])
              ),
    color = alt.Color('CONSUMPTION:Q', aggregate = 'average', legend=None),
    tooltip = [
    alt.Tooltip(title = 'К-сть енергії в МВт', field = 'CONSUMPTION', type = 'quantitative', aggregate = 'average', format = '.4s'),
    ],
).properties(width = 750, height = 500, background = '#F9F9F9', padding = 25
).configure_title(dy=-20, 
      color = textgrey, 
      fontWeight=600, 
      fontSize=14)